# Semantic Kernel dengan Integrasi Pelayan OpenBnB MCP

Notebook ini menunjukan cara menggunakan Semantic Kernel dengan pelayan OpenBnB MCP sebenar untuk mencari penginapan Airbnb nyata menggunakan MCPStdioPlugin. Untuk Akses LLM, ia menggunakan Microsoft Foundry. Untuk menyediakan pembolehubah persekitaran anda, anda boleh mengikuti [Pelajaran Persediaan](/00-course-setup/README.md)


## Import Pakej yang Diperlukan


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Mencipta Sambungan Pemalam MCP

Kita akan menyambung ke [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) menggunakan MCPStdioPlugin. Pelayan ini menyediakan fungsi carian Airbnb melalui pakej @openbnb/mcp-server-airbnb.


## Membuat Pelanggan

Dalam contoh ini, kami akan menggunakan Microsoft Foundry untuk akses LLM kami. Pastikan pembolehubah persekitaran anda ditetapkan dengan betul.


## Konfigurasi Persekitaran

Konfigurasikan tetapan Azure OpenAI. Pastikan anda mempunyai pemboleh ubah persekitaran berikut disetkan:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Memahami Integrasi OpenBnB MCP

Notebook ini menyambung ke **pelayan OpenBnB MCP sebenar** yang menyediakan fungsi carian Airbnb sebenar.

### Cara ia berfungsi:

1. **MCPStdioPlugin**: Menggunakan komunikasi input/output standard dengan pelayan MCP  
2. **Pakej NPM Sebenar**: Memuat turun dan menjalankan `@openbnb/mcp-server-airbnb` melalui npx  
3. **Data Langsung**: Mengembalikan data hartanah Airbnb sebenar dari API mereka  
4. **Penemuan Fungsi**: Ejen secara automatik menemui fungsi yang tersedia dari pelayan MCP  

### Fungsi Tersedia:

Pelayan OpenBnB MCP biasanya menyediakan:  
- **search_listings** - Cari hartanah Airbnb mengikut lokasi dan kriteria  
- **get_listing_details** - Dapatkan maklumat terperinci tentang hartanah tertentu  
- **check_availability** - Semak ketersediaan untuk tarikh tertentu  
- **get_reviews** - Dapatkan ulasan untuk hartanah  
- **get_host_info** - Dapatkan maklumat tentang hos hartanah  

### Prasyarat:

- **Node.js** dipasang pada sistem anda  
- **Sambungan Internet** untuk memuat turun pakej pelayan MCP  
- **NPX** tersedia (datang bersama Node.js)  

### Menguji Sambungan:

Anda boleh menguji pelayan MCP secara manual dengan menjalankan:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
Ini akan memuat turun dan memulakan pelayan OpenBnB MCP, yang kemudian disambungkan oleh Semantic Kernel untuk data Airbnb sebenar.


## Menjalankan Ejen dengan Pelayan MCP OpenBnB

Sekarang kami akan menjalankan Ejen AI yang bersambung ke pelayan MCP OpenBnB untuk mencari penginapan Airbnb sebenar di Stockholm untuk 2 dewasa dan 1 kanak-kanak. Sila ubah senarai `user_inputs` untuk mengubah kriteria carian.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Ringkasan
Tahniah! Anda berjaya membina ejen AI yang berintegrasi dengan carian penginapan dunia sebenar menggunakan Protokol Konteks Model (MCP):

## Teknologi yang Digunakan:
- Semantic Kernel - Untuk membina ejen pintar dengan Azure OpenAI
- Microsoft Foundry - Untuk keupayaan LLM dan pelengkap sembang
- MCP (Protokol Konteks Model) - Untuk integrasi alat berstandard
- OpenBnB MCP Server - Untuk fungsi carian Airbnb sebenar
- Node.js/NPX - Untuk menjalankan pelayan MCP luaran

## Apa yang Anda Pelajari:
- Integrasi MCP: Menghubungkan ejen Semantic Kernel dengan pelayan MCP luaran
- Akses Data Masa Nyata: Mencari hartanah Airbnb sebenar melalui API langsung
- Komunikasi Protokol: Menggunakan komunikasi stdio antara ejen dan pelayan MCP
- Penemuan Fungsi: Mengesan fungsi tersedia dari pelayan MCP secara automatik
- Penstriman Respons: Menangkap dan merekod panggilan fungsi secara masa nyata
- Render HTML: Memformat respons ejen dengan jadual bergaya dan paparan interaktif

## Langkah Seterusnya:
- Mengintegrasikan pelayan MCP tambahan (cuaca, penerbangan, restoran)
- Membina sistem berbilang ejen yang menggabungkan protokol MCP dan A2A
- Mencipta pelayan MCP tersuai untuk sumber data sendiri
- Melaksanakan memori perbualan berterusan merentasi sesi
- Melancarkan ejen ke Azure Functions dengan orkestrasi pelayan MCP
- Menambah pengesahan pengguna dan keupayaan tempahan


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya hendaklah dianggap sumber yang sahih. Untuk maklumat penting, terjemahan profesional oleh manusia adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
